In [156]:
import importlib
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta

In [157]:
import cafle as cf
from cafle.genfunc import rounding as R

In [158]:
# import cf.management as mng
from cafle import management_actualcase1 as mng1

In [159]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 200
print(pd.get_option("display.max_columns"), 
      pd.get_option("display.max_rows"))

40 200


# Introduction
* This is an example created by applying an actual case.

# Initial Setting

In [160]:
PY = 1/3.305785

In [161]:
class Assumption:
    def __init__(self):
        pass
    
    def __getattr__(self, attr):
        return self.__dict__[attr]
    
    def __getitem__(self, key):
        return self.__dict__[key]
astn = Assumption()

In [162]:
class Assumption_sales:
    def __init__(self):
        self.prdtname = []
        self.mtrx = DataFrame()
        self.csh = {}
        self.ctrt = {}
        self.ctrt_ary = {}
        
    def iptctrt(self, prdt_name, tmpary):
        tmp_idx = [x[0] for x in tmpary]
        tmp_locno = [idx.sales.locno(x[0]) for x in tmpary]
        tmp_rate = [x[1] for x in tmpary]
        self.ctrt_ary[prdt_name] = tmpary
        self.ctrt[prdt_name] = DataFrame({'locno':tmp_locno,
                                          'rate':tmp_rate},
                                          index=tmp_idx)
        self.ctrt[prdt_name]['amt'] = self.ctrt[prdt_name]['rate'] \
                                      * self.mtrx.amt[prdt_name]
        self.ctrt[prdt_name]['odr'] = list(range(len(tmpary)))
        self.ctrt[prdt_name] = self.ctrt[prdt_name][['odr', 'locno', 'rate', 'amt']]
        
    def print_ctrt(self):
        print("Sales Contract Assumption")
        for key, val in self.ctrt.items():
            print("\n\n"+key)
            display(R(val))

    def __getattr__(self, attr):
        return self.__dict__[attr]
    
    def __getitem__(self, key):
        return self.__dict__[key]
            
astns = Assumption_sales()

# Input Index Data

In [163]:
# Set period index
# prjt index : total project periods, and project base index
# cstrn index : construction index
# loan index : loan period index
# sales index : sales period index

tmp_col = ['idxname', 'start', 'periods']
tmp_ary = [['prjt',  '2021-04',   54],
           ['cstrn', '2022-03',   42],
           ['loan',  '2021-05',   55],
           ['sales', '2022-05',   40]]
astn.index = DataFrame(tmp_ary, columns=tmp_col)

idx = cf.PrjtIndex(idxname = list(astn.index.idxname),
                   start = list(astn.index.start),
                   periods = list(astn.index.periods + 1),
                   freq = 'M')

# Input Sales Data
### 1) Sales Amount Matrix

In [164]:
# Sales Amount Matrix
tmp_col = ['prdtname', 'area_m2',  'prc_py',  'byname']
tmp_ary = [['lgstc',    244_377,     1.40,   '물류시설용지'],
           ['cmcl',      34_926,     2.90,   '상류시설용지'],
           ['hsg_sgl',    9_137,     2.80,   '단독주택'],
           ['hsg_apt',   33_134,     2.70,   '공동주택'],
           ['retail',    17_195,     3.90,   '상업시설용지'],
           ['office',     6_151,     3.90,   '업무시설용지'],
           ['parking',    8_352,     1.90,   '주차장용지'],
           ['gasstn',     2_400,     2.60,   '주유소용지'],
           ['rcclg',     10_000,     2.80,   '자원순환시설'],
           ['power',     10_000,     2.80,   '발전시설용지'],
           ['gsprfr',        25,     1.44,   '가스정압기'],
           ]

tmp_df = DataFrame(tmp_ary, columns=tmp_col)
tmp_df.set_index('prdtname', inplace=True)
tmp_df.loc[:,'area_py'] = tmp_df.area_m2 * PY
tmp_df.loc[:,'prc_m2'] = tmp_df.prc_py / PY
tmp_df.loc[:,'amt'] = tmp_df.area_py * tmp_df.prc_py
tmp_df = tmp_df[['byname', 'area_m2', 'area_py', \
                'prc_m2', 'prc_py', 'amt']]

astns.prdtname = [x[0] for x in tmp_ary]
astns.mtrx = tmp_df
R(astns.mtrx)

,byname,area_m2,area_py,prc_m2,prc_py,amt
prdtname,,,,,,
lgstc,물류시설용지,"244,377","73,924",4.6,1.4,"103,494"
cmcl,상류시설용지,"34,926","10,565",9.6,2.9,"30,639"
hsg_sgl,단독주택,"9,137","2,764",9.3,2.8,"7,739"
hsg_apt,공동주택,"33,134","10,023",8.9,2.7,"27,062"
retail,상업시설용지,"17,195","5,201",12.9,3.9,"20,286"
office,업무시설용지,"6,151","1,861",12.9,3.9,"7,257"
parking,주차장용지,"8,352","2,526",6.3,1.9,"4,800"
gasstn,주유소용지,"2,400",726,8.6,2.6,"1,888"
rcclg,자원순환시설,"10,000","3,025",9.3,2.8,"8,470"


### 2) Sales Contract Schedule

In [165]:
# Sales Contract Schedule
prdt_name = 'lgstc' # 물류시설용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.315],
           [idx.sales.locval(2023, 5), 0.274],
           [idx.sales.locval(2023, 11), 0.137],
           [idx.sales.locval(2024, 5), 0.137],
           [idx.sales.locval(2024, 11), 0.137]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'cmcl' # 상류시설용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.150],
           [idx.sales.locval(2023, 5), 0.200],
           [idx.sales.locval(2023, 11), 0.300],
           [idx.sales.locval(2024, 5), 0.300],
           [idx.sales.locval(2024, 11), 0.050]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'hsg_sgl' # 단독주택용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.300],
           [idx.sales.locval(2023, 5), 0.300],
           [idx.sales.locval(2023, 11), 0.200],
           [idx.sales.locval(2024, 5), 0.200],
           [idx.sales.locval(2024, 11), 0.000]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'hsg_apt' # 공동주택용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.000],
           [idx.sales.locval(2023, 5), 0.000],
           [idx.sales.locval(2023, 11), 1.000],
           [idx.sales.locval(2024, 5), 0.000],
           [idx.sales.locval(2024, 11), 0.000]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'retail' # 상업시설용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.000],
           [idx.sales.locval(2023, 5), 0.250],
           [idx.sales.locval(2023, 11), 0.300],
           [idx.sales.locval(2024, 5), 0.300],
           [idx.sales.locval(2024, 11), 0.150]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'office' # 업무시설용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.000],
           [idx.sales.locval(2023, 5), 0.500],
           [idx.sales.locval(2023, 11), 0.500],
           [idx.sales.locval(2024, 5), 0.000],
           [idx.sales.locval(2024, 11), 0.000]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'parking' # 주차장용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.000],
           [idx.sales.locval(2023, 5), 0.000],
           [idx.sales.locval(2023, 11), 0.500],
           [idx.sales.locval(2024, 5), 0.500],
           [idx.sales.locval(2024, 11), 0.000]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'gasstn' # 주유소용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.000],
           [idx.sales.locval(2023, 5), 0.000],
           [idx.sales.locval(2023, 11), 1.000],
           [idx.sales.locval(2024, 5), 0.000],
           [idx.sales.locval(2024, 11), 0.000]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'rcclg' # 자원순환시설용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.000],
           [idx.sales.locval(2023, 5), 1.000],
           [idx.sales.locval(2023, 11), 0.000],
           [idx.sales.locval(2024, 5), 0.000],
           [idx.sales.locval(2024, 11), 0.000]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'power' # 발전시설용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.000],
           [idx.sales.locval(2023, 5), 1.000],
           [idx.sales.locval(2023, 11), 0.000],
           [idx.sales.locval(2024, 5), 0.000],
           [idx.sales.locval(2024, 11), 0.000]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'gsprfr' # 가스정압기용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.000],
           [idx.sales.locval(2023, 5), 0.000],
           [idx.sales.locval(2023, 11), 0.000],
           [idx.sales.locval(2024, 5), 0.000],
           [idx.sales.locval(2024, 11), 1.000]]
astns.iptctrt(prdt_name, tmp_ary)

### 3) Sales Payment Schedule

In [166]:
# Payment Schedule(deposit, middle payment, balance)
astns.csh[0] = [[idx.sales.locval(2022, 11), 0.1],
                [idx.sales.locval(2023,  5), 0.2],
                [idx.sales.locval(2023, 11), 0.2],
                [idx.sales.locval(2024,  5), 0.2],
                [idx.sales.locval(2024, 11), 0.2],
                [idx.sales.locval(2025,  9), 0.1]]
astns.csh[1] = [[idx.sales.locval(2023,  5), 0.1],
                [idx.sales.locval(2023, 10), 0.2],
                [idx.sales.locval(2024,  3), 0.2],
                [idx.sales.locval(2024,  8), 0.2],
                [idx.sales.locval(2025,  1), 0.2],
                [idx.sales.locval(2025,  9), 0.1]]
astns.csh[2] = [[idx.sales.locval(2023, 11), 0.1],
                [idx.sales.locval(2024,  3), 0.2],
                [idx.sales.locval(2024,  7), 0.2],
                [idx.sales.locval(2024, 11), 0.2],
                [idx.sales.locval(2025,  3), 0.2],
                [idx.sales.locval(2025,  9), 0.1]]
astns.csh[3] = [[idx.sales.locval(2024,  5), 0.1],
                [idx.sales.locval(2024,  8), 0.2],
                [idx.sales.locval(2024, 11), 0.2],
                [idx.sales.locval(2025,  2), 0.2],
                [idx.sales.locval(2025,  5), 0.2],
                [idx.sales.locval(2025,  9), 0.1]]
astns.csh[4] = [[idx.sales.locval(2024, 11), 0.1],
                [idx.sales.locval(2025,  1), 0.2],
                [idx.sales.locval(2025,  3), 0.2],
                [idx.sales.locval(2025,  5), 0.2],
                [idx.sales.locval(2025,  7), 0.2],
                [idx.sales.locval(2025,  9), 0.1]]

### 4) Initialize Sales Setting

In [167]:
sales = mng1.Intlz_sales_sellinlots(idx, idx.sales,
                                    slsmtrx=astns.mtrx,
                                    slsctrt=astns.ctrt,
                                    slscsh=astns.csh)

# Execution Cash Flow

In [168]:
acc = mng1.Intlz_accounts(idx, ['oprtg', 'sales', 'repay'])

In [169]:
# Execute cash flow
for idxno in idx.index:
    
    #### Sales ####
    # 분양수입대금 sales schedule에 따라 운영계좌로 이체
    for prdt in sales.prdtlst:
        sls_prdt = getattr(sales, prdt)
        mng_prdt = mng1.Mngmnt_sls(idxno, sls_prdt)
        mng_prdt.make_ctrt_plan()
        sls_amt = sls_prdt.sub_rsdl_cum[idxno]
        sls_prdt.send(idxno, sls_amt, acc.sales)
        
    
    

In [176]:
R(acc.sales.df)

,add_scdd,sub_scdd,bal_strt,amt_add,amt_sub,bal_end
2021-04-30,0.0,0.0,0,0,0.0,0
2021-05-31,0.0,0.0,0,0,0.0,0
2021-06-30,0.0,0.0,0,0,0.0,0
2021-07-31,0.0,0.0,0,0,0.0,0
2021-08-31,0.0,0.0,0,0,0.0,0
2021-09-30,0.0,0.0,0,0,0.0,0
2021-10-31,0.0,0.0,0,0,0.0,0
2021-11-30,0.0,0.0,0,0,0.0,0
2021-12-31,0.0,0.0,0,0,0.0,0
2022-01-31,0.0,0.0,0,0,0.0,0
